#### *setup (do not edit)*

In [1]:
#%pip install statbotics tbapy ipydatagrid pandas bqplot

In [2]:
# connect to APIs
import statbotics
import tbapy
sb = statbotics.Statbotics()
tba = tbapy.TBA("k8NOyJXkiEC85CiJDiHN4hanIXd1CpKshzbSXkTo0IDdhDZdrjjm2jC9GqtgAbBL")
assert(tba.status())

In [3]:
# types to help code hinting
from typing import TypedDict, Literal, Union

class StatboticsTeamMatch(TypedDict):
    team: int
    year: int
    event: str
    match: str
    alliance: Union[Literal["blue"], Literal["red"]]
    dq: bool
    epa: float
    auto_epa: float
    teleop_epa: float
    endgame_epa: float
    rp_1_epa: float
    rp_2_epa: float
    post_epa: float


class TBAEventMatchAlliance(TypedDict):
    dq_team_keys: list[str]
    score: int
    team_keys: list[str]

class TBAEventMatchAlliances(TypedDict):
    red: TBAEventMatchAlliance
    blue: TBAEventMatchAlliance

TBAScoreBreakdownCommunityRow = list[
    Union[Literal["None"], Literal["Cone"], Literal["Cube"]]
]

class TBAScoreBreakdownCommunity(TypedDict):
    B: TBAScoreBreakdownCommunityRow
    M: TBAScoreBreakdownCommunityRow
    T: TBAScoreBreakdownCommunityRow

TBAScoreBreakdownCommunityBridgeState = Union[Literal["Level"], Literal["NotLevel"]]
TBAScoreBreakdownCommunityRobotDockState = Union[Literal["Docked"], Literal["None"]]
TBAScoreBreakdownMobility = Union[Literal["Yes"], Literal["No"]]

class TBAScoreBreakdownLink(TypedDict):
    nodes: list[int]
    row: Union[Literal["Bottom"], Literal["Mid"], Literal["Top"]]

class TBAScoreBreakdown(TypedDict):
    activationBonusAchieved: bool
    autoBridgeState: TBAScoreBreakdownCommunityBridgeState
    autoChargeStationPoints: int
    autoDocked: bool
    autoGamePieceCount: int
    autoGamePiecePoints: int
    autoMobilityPoints: int
    autoPoints: int
    coopGamePieceCount: int
    coopertitionCriteriaMet: bool
    endGameBridgeState: TBAScoreBreakdownCommunityBridgeState
    endGameChargeStationPoints: int
    endGameChargeStationRobot1: TBAScoreBreakdownCommunityRobotDockState
    endGameChargeStationRobot2: TBAScoreBreakdownCommunityRobotDockState
    endGameChargeStationRobot3: TBAScoreBreakdownCommunityRobotDockState
    endGameParkPoints: int
    foulCount: int
    foulPoints: int
    linkPoints: int
    links: list[TBAScoreBreakdownLink]
    mobilityRobot1: TBAScoreBreakdownMobility
    mobilityRobot2: TBAScoreBreakdownMobility
    mobilityRobot3: TBAScoreBreakdownMobility
    rp: int
    sustainabilityBonusAchieved: bool
    techFoulCount: int
    teleopCommunity: TBAScoreBreakdownCommunity
    teleopGamePieceCount: int
    teleopGamePiecePoints: int
    teleopPoints: int
    totalChargeStationPoints: int
    totalPoints: int
    autoChargeStationRobot1: TBAScoreBreakdownCommunityRobotDockState
    autoChargeStationRobot2: TBAScoreBreakdownCommunityRobotDockState
    autoChargeStationRobot3: TBAScoreBreakdownCommunityRobotDockState
    autoCommunity: TBAScoreBreakdownCommunity

class TBAScoreBreakdowns(TypedDict):
    red: TBAScoreBreakdown
    blue: TBAScoreBreakdown

class TBAVideoEntry(TypedDict):
    type: Union[Literal["youtube"], Literal["tba"]]
    key: str

class TBAEventMatch(TypedDict):
    key: str
    alliances: TBAEventMatchAlliances
    score_breakdown: TBAScoreBreakdowns
    videos: list[TBAVideoEntry]
    winning_alliance: Union[Literal["red"], Literal["blue"]]
    event_key: str
    actual_time: int


In [4]:
# preload data for competition
YEAR = 2023
EVENT = "2023cafr"

teams_for_event: list[int] = [t["team_number"] for t in tba.event_teams(event=EVENT)]
prior_events_for_teams: set[str] = set()
prior_sb_team_matches: list[StatboticsTeamMatch] = []
prior_tba_event_matches: list[TBAEventMatch] = []
for team in teams_for_event:
    events = tba.team_events(team, year=YEAR)
    for event in events:
        event_occurred = bool(event["webcasts"])
        event_is_not_this_event = event["key"] != EVENT
        if event_occurred and event_is_not_this_event:
            prior_events_for_teams.add(event["key"])
for prior_event in prior_events_for_teams:
    prior_sb_team_matches += sb.get_team_matches(event=prior_event, limit=600) # type: ignore
    prior_tba_event_matches += tba.event_matches(event=prior_event)

print(f"{len(teams_for_event)} teams going to {EVENT}")
print(f"{len(prior_tba_event_matches)} prior matches we can use for scouting")

38 teams going to 2023cafr
202 prior matches we can use for scouting


In [5]:
# helpers
import json

def show(obj) -> None:
    print(json.dumps(obj, indent=4))

def save_data(name: str, data: Union[dict, list]) -> None:
    with open(f".data/{name}.json", "w") as f:
        json.dump(data, f)

def load_data(name: str) -> Union[dict, list]:
    with open(f".data/{name}.json", "r") as f:
        return json.load(f)

In [6]:
save_data("prior_sb_team_matches", prior_sb_team_matches)
save_data("prior_tba_event_matches", prior_tba_event_matches)
save_data("teams_for_event", teams_for_event)

# Process Latest Event Data

* Use `tba` for [BlueAlliance data](https://github.com/frc1418/tbapy)
* Use `sb` for [Statbotics data](https://statbotics.readthedocs.io/en/latest/)

In [7]:
# re-run this cell whenever you need updated data for the current event
sb_team_matches: list[StatboticsTeamMatch] = prior_sb_team_matches + sb.get_team_matches(event=EVENT, limit=600) # type: ignore
tba_event_matches: list[TBAEventMatch] = prior_tba_event_matches + tba.event_matches(event=EVENT)

save_data("sb_team_matches", sb_team_matches)
save_data("tba_event_matches", tba_event_matches)

print(f"{len(teams_for_event)} teams going to {EVENT}")
print(f"{len(sb_team_matches)} match entries from Statbotics")
print(f"{len(tba_event_matches)} match entries from TheBlueAlliance")

38 teams going to 2023cafr
1740 match entries from Statbotics
294 match entries from TheBlueAlliance


In [8]:
# extract the data (from both TBA and Statbotics) flattened into per-match
from collections import defaultdict
from typing import NamedTuple, Optional

def extract_team_numbers(alliance: TBAEventMatchAlliance) -> list[int]:
    return [int(k.replace("frc", "")) for k in alliance["team_keys"]]

def remap_to_matches(matches: list[StatboticsTeamMatch]) -> dict[str, list[StatboticsTeamMatch]]:
    mapped: dict[str, list[StatboticsTeamMatch]] = defaultdict(list)
    for m in matches:
        match_key = m["match"]
        mapped[match_key].append(m)
    return mapped

def count_matching_values(iterable: list[str], str_to_find: str) -> int:
    return len([i for i in iterable if i == str_to_find])

Alliance = Union[Literal["red"], Literal["blue"]]

class MatchRow(NamedTuple):
    match_key: str
    youtube_url: Optional[str]
    team: int
    alliance_score: int
    opponent_score: int
    ranking_points: int
    epa_after: float
    opr_overall: Optional[float]
    opr_grid_bottom: Optional[float]
    opr_grid_mid: Optional[float]
    opr_grid_top: Optional[float]
    opr_links: Optional[float]
    opr_charge_station: Optional[float]
    opr_cubes: Optional[float]
    opr_cones: Optional[float]
    auto_epa: float
    teleop_epa: float
    endgame_epa: float
    links_rp_epa: float
    activation_rp_epa: float
    link_count: int
    grid_auto_bottom: int
    grid_auto_mid: int
    grid_auto_top: int
    grid_teleop_bottom: int
    grid_teleop_mid: int
    grid_teleop_top: int
    cones_scored: int
    cubes_scored: int
    activated_in_auto: bool
    activated_in_endgame: bool
    coopertition: bool
    alliance_drew_fouls: int
    alliance_committed_fouls: int
    failed_activation_in_endgame: bool
    failed_mobility_in_auto: bool
    failed_activation_in_auto: bool
    charge_station_points: int
    disqualified: bool
    underperformed: bool
    epa_before: float
    event: str
    winner: bool
    alliance: Alliance
    alliance_partner1: int
    alliance_partner2: int
    alliance_partner1_epa: float
    alliance_partner2_epa: float
    opponent1: int
    opponent2: int
    opponent3: int
    opponent1_epa: float
    opponent2_epa: float
    opponent3_epa: float
    timestamp: int
    # TODO: could you make an OPR of the grid scores?
    # TODO: could you make an OPR of the link scores?

final_rows: list[MatchRow] = []
mapped_matches = remap_to_matches(sb_team_matches)

for tba_event_match in tba_event_matches:
    event_key = tba_event_match["event_key"]
    match_key = tba_event_match["key"]
    team_matches = mapped_matches[match_key]
    blue_teams = extract_team_numbers(tba_event_match["alliances"]["blue"])
    red_teams = extract_team_numbers(tba_event_match["alliances"]["red"])

    endgame_charger_by_team: dict[int, TBAScoreBreakdownCommunityRobotDockState] = {}
    autonomous_charger_by_team: dict[int, TBAScoreBreakdownCommunityRobotDockState] = {}
    mobility_by_team: dict[int, TBAScoreBreakdownMobility] = {}

    score_breakdown = tba_event_match["score_breakdown"]
    match_did_not_occur = not score_breakdown
    if match_did_not_occur:
        continue

    autonomous_charger_by_team[blue_teams[0]] = score_breakdown["blue"]["autoChargeStationRobot1"]
    autonomous_charger_by_team[blue_teams[1]] = score_breakdown["blue"]["autoChargeStationRobot2"]
    autonomous_charger_by_team[blue_teams[2]] = score_breakdown["blue"]["autoChargeStationRobot3"]
    autonomous_charger_by_team[red_teams[0]] = score_breakdown["red"]["autoChargeStationRobot1"]
    autonomous_charger_by_team[red_teams[1]] = score_breakdown["red"]["autoChargeStationRobot2"]
    autonomous_charger_by_team[red_teams[2]] = score_breakdown["red"]["autoChargeStationRobot3"]

    endgame_charger_by_team[blue_teams[0]] = score_breakdown["blue"]["endGameChargeStationRobot1"]
    endgame_charger_by_team[blue_teams[1]] = score_breakdown["blue"]["endGameChargeStationRobot2"]
    endgame_charger_by_team[blue_teams[2]] = score_breakdown["blue"]["endGameChargeStationRobot3"]
    endgame_charger_by_team[red_teams[0]] = score_breakdown["red"]["endGameChargeStationRobot1"]
    endgame_charger_by_team[red_teams[1]] = score_breakdown["red"]["endGameChargeStationRobot2"]
    endgame_charger_by_team[red_teams[2]] = score_breakdown["red"]["endGameChargeStationRobot3"]

    mobility_by_team[blue_teams[0]] = score_breakdown["blue"]["mobilityRobot1"]
    mobility_by_team[blue_teams[1]] = score_breakdown["blue"]["mobilityRobot2"]
    mobility_by_team[blue_teams[2]] = score_breakdown["blue"]["mobilityRobot3"]
    mobility_by_team[red_teams[0]] = score_breakdown["red"]["mobilityRobot1"]
    mobility_by_team[red_teams[1]] = score_breakdown["red"]["mobilityRobot2"]
    mobility_by_team[red_teams[2]] = score_breakdown["red"]["mobilityRobot3"]

    all_teams = blue_teams + red_teams
    for team in all_teams:
        for team_match in team_matches:
            if team_match["team"] == team:

                alliance_teams = blue_teams if team in blue_teams else red_teams
                opponent_teams = red_teams if team in blue_teams else blue_teams
                alliance_breakdown = score_breakdown["blue"] if team in blue_teams else score_breakdown["red"]
                opponent_breakdown = score_breakdown["red"] if team in blue_teams else score_breakdown["blue"]

                was_level_in_auto = alliance_breakdown["autoBridgeState"] == "Level"
                was_level_in_endgame = alliance_breakdown["endGameBridgeState"] == "Level"

                alliance_committed_fouls = alliance_breakdown["foulCount"]
                alliance_drew_fouls = int(alliance_breakdown["foulPoints"] / 5)
                link_count = len(alliance_breakdown["links"])

                grid_auto_bottom_cubes = count_matching_values(alliance_breakdown["autoCommunity"]["B"], "Cube")
                grid_auto_mid_cubes = count_matching_values(alliance_breakdown["autoCommunity"]["M"], "Cube")
                grid_auto_top_cubes = count_matching_values(alliance_breakdown["autoCommunity"]["T"], "Cube")

                grid_teleop_bottom_cubes = count_matching_values(alliance_breakdown["teleopCommunity"]["B"], "Cube")
                grid_teleop_mid_cubes = count_matching_values(alliance_breakdown["teleopCommunity"]["M"], "Cube")
                grid_teleop_top_cubes = count_matching_values(alliance_breakdown["teleopCommunity"]["T"], "Cube")

                grid_auto_bottom_cones = count_matching_values(alliance_breakdown["autoCommunity"]["B"], "Cone")
                grid_auto_mid_cones = count_matching_values(alliance_breakdown["autoCommunity"]["M"], "Cone")
                grid_auto_top_cones = count_matching_values(alliance_breakdown["autoCommunity"]["T"], "Cone")

                grid_teleop_bottom_cones = count_matching_values(alliance_breakdown["teleopCommunity"]["B"], "Cone")
                grid_teleop_mid_cones = count_matching_values(alliance_breakdown["teleopCommunity"]["M"], "Cone")
                grid_teleop_top_cones = count_matching_values(alliance_breakdown["teleopCommunity"]["T"], "Cone")

                grid_auto_bottom = grid_auto_bottom_cubes + grid_auto_bottom_cones
                grid_auto_mid = grid_auto_mid_cubes + grid_auto_mid_cones
                grid_auto_top = grid_auto_top_cubes + grid_auto_top_cones

                grid_teleop_bottom = grid_teleop_bottom_cubes + grid_teleop_bottom_cones
                grid_teleop_mid = grid_teleop_mid_cubes + grid_teleop_mid_cones
                grid_teleop_top = grid_teleop_top_cubes + grid_teleop_top_cones

                coopertition = alliance_breakdown["coopertitionCriteriaMet"]
                ranking_points = alliance_breakdown["rp"]

                alliance_partners = list(set(alliance_teams) - set([team]))

                alliance_partner1 = alliance_partners[0]
                alliance_partner1_epa = [t["epa"] for t in team_matches if t["team"] == alliance_partner1][0]
                alliance_partner2 = alliance_partners[1]
                alliance_partner2_epa = [t["epa"] for t in team_matches if t["team"] == alliance_partner2][0]

                opponent1 = opponent_teams[0]
                opponent1_epa = [t["epa"] for t in team_matches if t["team"] == opponent1][0]
                opponent2 = opponent_teams[1]
                opponent2_epa = [t["epa"] for t in team_matches if t["team"] == opponent2][0]
                opponent3 = opponent_teams[2]
                opponent3_epa = [t["epa"] for t in team_matches if t["team"] == opponent3][0]

                activated_in_auto = autonomous_charger_by_team[team] == "Docked" and was_level_in_auto
                activated_in_endgame = endgame_charger_by_team[team] == "Docked" and was_level_in_endgame

                failed_activation_in_auto = autonomous_charger_by_team[team] == "Docked" and not was_level_in_auto
                failed_activation_in_endgame = endgame_charger_by_team[team] == "Docked" and not was_level_in_endgame

                youtube_videos = [w for w in tba_event_match["videos"] if w["type"] == "youtube"]
                youtube_url = f"https://www.youtube.com/watch?v={youtube_videos[0]['key']}" if youtube_videos else None

                row = MatchRow(
                    timestamp=tba_event_match["actual_time"],
                    team=team,
                    event=event_key.replace(str(YEAR), ""),
                    match_key=match_key.replace(str(YEAR), ""),
                    winner=tba_event_match["winning_alliance"] == ( "blue" if team in blue_teams else "red" ),
                    underperformed=team_match["post_epa"] < (0.95 * team_match["epa"]),
                    alliance="blue" if team in blue_teams else "red",
                    alliance_score=alliance_breakdown["totalPoints"],
                    opponent_score=opponent_breakdown["totalPoints"],
                    disqualified=team_match["dq"],
                    epa_before=team_match["epa"],
                    epa_after=team_match["post_epa"],
                    charge_station_points=alliance_breakdown["totalChargeStationPoints"],
                    opr_overall=None,
                    opr_grid_bottom=None,
                    opr_grid_mid=None,
                    opr_grid_top=None,
                    opr_links=None,
                    opr_charge_station=None,
                    opr_cubes=None,
                    opr_cones=None,
                    auto_epa=team_match["auto_epa"],
                    teleop_epa=team_match["teleop_epa"],
                    endgame_epa=team_match["endgame_epa"],
                    links_rp_epa=team_match["rp_1_epa"],
                    activation_rp_epa=team_match["rp_2_epa"],
                    activated_in_auto=activated_in_auto,
                    activated_in_endgame=activated_in_endgame,
                    failed_activation_in_auto=failed_activation_in_auto,
                    failed_activation_in_endgame=failed_activation_in_endgame,
                    failed_mobility_in_auto=mobility_by_team[team] != "Yes",
                    alliance_committed_fouls=alliance_committed_fouls,
                    alliance_drew_fouls=alliance_drew_fouls,
                    link_count=link_count,
                    cones_scored=grid_auto_bottom_cones + grid_auto_mid_cones + grid_auto_top_cones + grid_teleop_bottom_cones + grid_teleop_mid_cones + grid_teleop_top_cones,
                    cubes_scored=grid_auto_bottom_cubes + grid_auto_mid_cubes + grid_auto_top_cubes + grid_teleop_bottom_cubes + grid_teleop_mid_cubes + grid_teleop_top_cubes,
                    grid_auto_bottom=grid_auto_bottom,
                    grid_auto_mid=grid_auto_mid,
                    grid_auto_top=grid_auto_top,
                    grid_teleop_bottom=grid_teleop_bottom,
                    grid_teleop_mid=grid_teleop_mid,
                    grid_teleop_top=grid_teleop_top,
                    coopertition=coopertition,
                    ranking_points=ranking_points,
                    alliance_partner1=alliance_partner1,
                    alliance_partner2=alliance_partner2,
                    alliance_partner1_epa=alliance_partner1_epa,
                    alliance_partner2_epa=alliance_partner2_epa,
                    opponent1=opponent1,
                    opponent1_epa=opponent1_epa,
                    opponent2=opponent2,
                    opponent2_epa=opponent2_epa,
                    opponent3=opponent3,
                    opponent3_epa=opponent3_epa,
                    youtube_url=youtube_url,
                )
                final_rows.append(row)


In [9]:
# calculate OPRs ongoing
from typing import Callable
import numpy as np

ScoreExtractor = Callable[[MatchRow], Union[int, float]]
ContributionMap = dict[int, float]

def calculate_power_ratings(match_rows: list[MatchRow], extract_score: ScoreExtractor) -> ContributionMap:

    unique_teams: set[int] = set()
    matches_by_key: dict[str, MatchRow] = {}
    for match_idx in match_rows:
        unique_teams.add(match_idx.team)
        unique_teams.add(match_idx.alliance_partner1)
        unique_teams.add(match_idx.alliance_partner2)
        matches_by_key[match_idx.match_key] = match_idx

    teams = list(unique_teams)
    match_keys = list(matches_by_key.keys())

    team_count = len(teams)
    match_count = len(match_keys)

    matches = np.zeros((match_count, team_count))
    scores = np.zeros(match_count)

    for match_idx in range(match_count):

        match_key = match_keys[match_idx]
        match = matches_by_key[match_key]
        score = extract_score(match)

        scores[match_idx] = score

        matches[match_idx][teams.index(match.team)] = 1
        matches[match_idx][teams.index(match.alliance_partner1)] = 1
        matches[match_idx][teams.index(match.alliance_partner2)] = 1

    try:
        contribution_to_score = np.linalg.solve(
            np.dot(matches.T, matches),
            np.dot(matches.T, scores),
        )
        contribution_to_score_truncated_significant_digits = [int(s * 100) / 100 for s in contribution_to_score]
        return dict(zip(teams, contribution_to_score_truncated_significant_digits))
    except np.linalg.LinAlgError:
        # Recurse on the previous matches since this is an unsolvable matrix for now.
        # TODO: use least squares to solve approximately?
        # https://blog.thebluealliance.com/2017/10/05/the-math-behind-opr-an-introduction/
        if len(match_rows) > 1:
            return {} #calculate_power_ratings(match_rows[:-1], extract_score)
        else:
            return {}

sorted_match_rows: list[MatchRow] = sorted(final_rows, key=lambda r: r.timestamp)
match_rows_with_oprs: list[MatchRow] = []
for i, match_row in enumerate(sorted_match_rows):
    rows_up_to_this_point = sorted_match_rows[:i+1]
    match_row_as_dict = match_row._asdict()
    # TODO: is there a typesafe way of doing this? with _replace()?
    match_row_as_dict["opr_overall"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.alliance_score).get(match_row.team, None)
    match_row_as_dict["opr_grid_bottom"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.grid_auto_bottom + r.grid_teleop_bottom).get(match_row.team, None)
    match_row_as_dict["opr_grid_mid"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.grid_auto_mid + r.grid_teleop_mid).get(match_row.team, None)
    match_row_as_dict["opr_grid_top"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.grid_auto_top + r.grid_teleop_top).get(match_row.team, None)
    match_row_as_dict["opr_links"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.link_count).get(match_row.team, None)
    match_row_as_dict["opr_charge_station"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.charge_station_points).get(match_row.team, None)
    match_row_as_dict["opr_cubes"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.cubes_scored).get(match_row.team, None)
    match_row_as_dict["opr_cones"] = calculate_power_ratings(rows_up_to_this_point,
        lambda r: r.cones_scored).get(match_row.team, None)
    match_rows_with_oprs.append(MatchRow(**match_row_as_dict))

final_rows = match_rows_with_oprs

In [10]:
# Prepare the final querying and rendering
# https://github.com/bloomberg/ipydatagrid/blob/main/examples/DataGrid.ipynb
# https://bqplot.readthedocs.io/en/stable/_generate/bqplot.scales.ColorScale.html
import pandas as pd
from ipydatagrid import DataGrid, BarRenderer, TextRenderer, Expr, HyperlinkRenderer
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale
from ipywidgets import interact

# CoLab requires this to render anything outside of its normal stuff
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pass

matchdata = pd.DataFrame(final_rows)
matchdata.set_index("timestamp")

render_team_number = TextRenderer(font=Expr("'16px sans-serif'"))
render_achieved_action = TextRenderer(background_color=Expr("'green' if cell.value else default_value"))
render_failed_action = TextRenderer(background_color=Expr("'red' if cell.value else default_value"))
render_video_link = HyperlinkRenderer(
    url=Expr("cell.value"),
    url_name=Expr("'watch'"),
    background_color="#ddd",
    text_color="blue",
    font="bold 14px Arial, sans-serif",
)

def create_heat_renderer(series: pd.Series, mid=None, reversed=False):
    max_value = series.astype(float).quantile(0.95)
    min_value = series.astype(float).quantile(0.05)
    mid_value = series.astype(float).quantile(0.50) if mid is None else mid
    return BarRenderer(
        horizontal_alignment="center",
        bar_color=ColorScale(min=min_value, mid=mid_value, max=max_value, scheme="RdYlGn", reverse=reversed),
        bar_value=LinearScale(min=min_value, mid=mid_value, max=max_value),
    )

def create_churro_scout_grid(selected_matchdata: pd.DataFrame) -> DataGrid:
    final_data = selected_matchdata.set_index("match_key")
    sorted_data = final_data.sort_values(by="timestamp") # type:ignore
    grid = DataGrid(sorted_data, renderers={
        "ranking_points": create_heat_renderer(matchdata.ranking_points),
        "youtube_url": render_video_link,
        # TODO: render scores a bit differently
        # "alliance_score": create_heat_renderer(matchdata.alliance_score),
        # "opponent_score": create_heat_renderer(matchdata.opponent_score, reversed=True),
        # "epa_before": create_heat_renderer(matchdata.epa_before),
        "epa_after": create_heat_renderer(matchdata.epa_after),
        "opr_overall": create_heat_renderer(matchdata.opr_overall),
        "opr_grid_bottom": create_heat_renderer(matchdata.opr_grid_bottom),
        "opr_grid_mid": create_heat_renderer(matchdata.opr_grid_mid),
        "opr_grid_top": create_heat_renderer(matchdata.opr_grid_top),
        "opr_links": create_heat_renderer(matchdata.opr_links),
        "opr_charge_station": create_heat_renderer(matchdata.opr_charge_station),
        "opr_cones": create_heat_renderer(matchdata.opr_cones),
        "opr_cubes": create_heat_renderer(matchdata.opr_cubes),
        "auto_epa": create_heat_renderer(matchdata.auto_epa),
        "teleop_epa": create_heat_renderer(matchdata.teleop_epa),
        "endgame_epa": create_heat_renderer(matchdata.endgame_epa),
        "links_rp_epa": create_heat_renderer(matchdata.links_rp_epa, mid=0),
        "activation_rp_epa": create_heat_renderer(matchdata.activation_rp_epa),
        "link_count": create_heat_renderer(matchdata.link_count),
        "grid_auto_bottom": create_heat_renderer(matchdata.grid_auto_bottom),
        "grid_auto_mid": create_heat_renderer(matchdata.grid_auto_mid),
        "grid_auto_top": create_heat_renderer(matchdata.grid_auto_top),
        "grid_teleop_bottom": create_heat_renderer(matchdata.grid_teleop_bottom),
        "grid_teleop_mid": create_heat_renderer(matchdata.grid_teleop_mid),
        "grid_teleop_top": create_heat_renderer(matchdata.grid_teleop_top),
        "coopertition": render_achieved_action,
        "activated_in_auto": render_achieved_action,
        "activated_in_endgame": render_achieved_action,
        "alliance_drew_fouls": create_heat_renderer(matchdata.alliance_drew_fouls),
        "alliance_committed_fouls": create_heat_renderer(matchdata.alliance_committed_fouls, reversed=True),
        "failed_mobility_in_auto": render_failed_action,
        "failed_activation_in_auto": render_failed_action,
        "failed_activation_in_endgame": render_failed_action,
        "disqualified": render_failed_action,
        "underperformed": render_failed_action,
        "team": render_team_number,
        "alliance_partner1": render_team_number,
        "alliance_partner2": render_team_number,
        "opponent1": render_team_number,
        "opponent2": render_team_number,
        "opponent3": render_team_number,
    })
    grid.auto_fit_columns = True
    return grid

def render_grid_for_team(team: str):
    filtered_data = matchdata[matchdata.team == int(team)]
    grid = create_churro_scout_grid(filtered_data)
    display(grid)

def churroscout():
    default_team = "8048"
    interact(render_grid_for_team, team=default_team)

# sample rendering for testing out the render settings
render_grid_for_team("8048")

DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_render…

# ChurroScout v1

In [11]:
churroscout()
# TODO: figure out why opr_grid_bottom is only 3.82 for us

interactive(children=(Text(value='8048', description='team'), Output()), _dom_classes=('widget-interact',))

In [125]:
import plotly.express as px
import plotly.graph_objects as go

# Aggregate data by the teams who are attending this event, looking at their
# 75th percentile performance in all categories. We normalize everything to
# a percentile so that we can compare teams across categories on a scatterplot.
ranked_team_data = matchdata.copy()
ranked_team_data = ranked_team_data[ranked_team_data.team.isin(teams_for_event)]
ranked_team_data = ranked_team_data.groupby("team").quantile(0.75, numeric_only=True).rank(pct=True)
scatter_labels = {
    8048: "churrobots",
    5940: "BREAD",
    # 973: "Greybots",
    # 1671: "Birdbrains",
    # 1323: "MadTown",
}
ranked_team_data["scatter_label"] = ranked_team_data.apply(lambda row: scatter_labels.get(row.name, "other"), axis=1)

# Render the scatterplot.
express_fig = px.scatter(
    ranked_team_data,
    x=ranked_team_data.opr_cones,
    y=ranked_team_data.opr_grid_top,
    size=ranked_team_data.opr_charge_station,
    hover_name=ranked_team_data.index,
    hover_data=[ranked_team_data.epa_after],
    color=ranked_team_data.scatter_label,
    width=800,
    height=500,
)

# Create a grid to show the selected team
filtered_data = matchdata[matchdata.team == 8048]
grid = create_churro_scout_grid(filtered_data)

# clicks update the grid to show the clicked team
def handle_clicked_point(trace, points, state):
    if points.point_inds:
        team_number_idx = points.point_inds[0]
        team_number = int(trace["hovertext"][team_number_idx])
        grid.data = matchdata[matchdata.team == team_number]

# Convert to a full fledged FigureWidget so we can get clicks
fig = go.FigureWidget(express_fig.data, express_fig.layout)
fig.data[0].on_click(handle_clicked_point) # type: ignore
fig.data[1].on_click(handle_clicked_point) # type: ignore
fig.data[2].on_click(handle_clicked_point) # type: ignore

# Do rendering
# fig.show(renderer="vscode")
display(fig)
display(grid)
# dh = display(display_id=True)

FigureWidget({
    'data': [{'customdata': array([[0.63157895],
                                   [0.84210526],
                                   [0.07894737],
                                   [0.94736842],
                                   [0.76315789],
                                   [0.73684211],
                                   [1.        ],
                                   [0.18421053],
                                   [0.92105263],
                                   [0.10526316],
                                   [0.52631579],
                                   [0.5       ],
                                   [0.15789474],
                                   [0.86842105],
                                   [0.02631579],
                                   [0.71052632],
                                   [0.68421053],
                                   [0.89473684],
                                   [0.65789474],
                                   [0.60526316],
     

DataGrid(auto_fit_columns=True, auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_render…